viernes 03 de mayo de 2019  
  
_Benjamín Hernández Cortés_ - _Juan Pablo Rojas Rojas_  
_Departamento de Ingeniería Informática (DIINF)_  
_Universidad de Santiago de Chile (USACH)_


## Laboratorio 2 - Fundamentos de Aprendizaje Profundo con Redes Neuronales
___

El presente código está orientado hacia la implementación de un perceptrón multicapa o red neuronal de múltiples capas (Multi-Layer Neural Network). En primera oportunidad, se utilizará el perceptrón para clasificar y emular compuertas lógicas AND, OR y XOR, tanto de 2 como de 4 entradas. Luego, se realizará una clasificación trabajando con el conjunto de datos [wine](https://archive.ics.uci.edu/ml/datasets/wine) provenientes de la [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) que contiene 13 atributos en total y que intentan describir clases de vinos provenientes de 3 cultivos distintos.


#### Importación de bibliotecas
---

Las bibliotecas a emplear son:
- **Numpy:** Herramienta de computación científica, que nos permitirá trabajar a través de vectores
- **Pandas:** Para la manipulación y lectura de datos
- **Matplotlib:** Para la visualización gráfica de diversos datos de interés
- **Itertools:** Como herramienta para iteración de objetos
- **Scikit-learn:** Para la obtención del dataset _wine_ y el uso de herramientas de evaluación de desempeño para los perceptrones multicapa.

In [83]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import itertools as its
import sklearn as skl
from sklearn.datasets import load_wine
from sklearn import preprocessing

#### Definición de la clase TruthTableGenerator
---

La clase TruthTableGenerator permite generar tablas de verdad para 3 tipos de compuertas lógicas: AND, OR y XOR.

En cuanto a las funciones definidas, se tiene:

| **Función**  | **Descripción**  |
| ------------ | ------------ |
| `generate_table(n_inputs, logic_gate)`  |  Genera una tabla de verdad de n-variables (*n_inputs*), basada en una compuerta lógica determinada (*logic_gate*)|

In [84]:
class TruthTableGenerator():
    
    def generate_table(n_inputs, logic_gate):
        table = its.product([0,1], repeat = n_inputs)
        table = pd.DataFrame(table)
        results = []
        
        if logic_gate == 'AND':
            for i in range(n_inputs ** 2):
                row = table.loc[[i]].values[0]
                results.append( int(all(row)) )
            
        elif logic_gate == 'OR':
            for i in range(n_inputs ** 2):
                row = table.loc[[i]].values[0]
                results.append( int(any(row)) )
                    
        elif logic_gate == 'XOR':
            for i in range(n_inputs ** 2):
                row = table.loc[[i]].values[0]
                tmp_result = row[0] ^ row[1]
                for j in range(2, n_inputs):
                    tmp_result = tmp_result ^ row[j]
                results.append( tmp_result )
        
        else:
            return None
        
        table['result'] = results
        return table

#### Definición de la clase Layer
---
La clase Layer permite representar una de las capas que conforma al Perceptrón Multicapa. Requiere de 3 parámetros:

| <p style='text-align: left;'>**Parámetro**</p> | <p style='text-align: left;'>**Descripción**</p> |
| ------------ | ------------ |
| `number_of_inputs` |  <p style='text-align: justify;'>Valor númerico entero. Define la cantidad de entradas que recibe la capa, la cual debe ser mayor que 0. Por defecto, se define el número de entradas como 1.</p>|
| `number_of_neurons` | <p style='text-align: justify;'>Valor numérico entero. Define la cantidad de neuronas que posee la capa, la cual debe ser mayor que 0. Por defecto, se define el número de neuronas como 1.</p> |
| `activation_function` | <p style='text-align: justify;'>Cadena de caracteres. Define la función de activación que empleará la capa, la cual puede ser sigmoide (_sigmoid_) o ReLU (_relu_).</p> |

En cuanto a las funciones definidas, se tiene:

| <p style='text-align: left;'>**Función**</p>  | <p style='text-align: left;'>**Descripción**</p> |
| ------------ | ------------ |
| `calculate_output(input_vector)` |  <p style='text-align: justify;'>Realiza una predicción para un conjunto de datos (*input_vector*), empleando los valores actuales de los pesos (*weights*) asociados a la capa.</p>|


In [128]:
class Layer:
    def __init__(self,
                 number_of_inputs = 1,
                 number_of_neurons = 1,
                 activation_function = 'sigmoid'):
        self.number_of_neurons = number_of_neurons
        self.number_of_inputs = number_of_inputs
        self.activation_function = activation_function
        self.weights = np.random.rand(self.number_of_inputs + 1, self.number_of_neurons)
        self.input = None
        self.output = None
        
    def calculate_output(self, input_vector):
        self.input = np.append(input_vector, 1)
        sum_ = np.matmul(self.input, self.weights)
        if self.activation_function == 'sigmoid': self.output = sigmoid(sum_)
        elif self.activation_function == 'relu': self.output = relu(sum_)
        else: self.output = None
        return self.output
        
            

class MLP:
    def __init__(self,
                 data = None,
                 number_of_inputs = 1,
                 number_of_hidden_layers = 0,
                 number_of_neurons_for_each_layer = None,
                 activation_functions_for_each_layer = None):
        
        self.data = data
        self.data_nrows, self.data_ncols = data.shape
        self.number_of_hidden_layers = number_of_hidden_layers
        self.number_of_neurons_for_each_layer = number_of_neurons_for_each_layer
        self.activation_functions_for_each_layer = activation_functions_for_each_layer
        self.layers = []
        
        # Primero se crea una capa escondida (hidden layer) según la cantidad de entradas
        # definidas para el perceptrón. Si no se define un número de capas escondidas,
        # entonces solamente se creará la capa de salida (output layer), recreando a un
        # perceptrón simple.
        
        self.layers.append(
            Layer(number_of_inputs = number_of_inputs,
                  number_of_neurons = number_of_neurons_for_each_layer[0],
                  activation_function = activation_functions_for_each_layer[0])
        )
        for i in range(1, self.number_of_hidden_layers + 1):
            self.layers.append(
                Layer(number_of_inputs = number_of_neurons_for_each_layer[i-1],
                      number_of_neurons = number_of_neurons_for_each_layer[i],
                      activation_function = activation_functions_for_each_layer[i])
            )
    
    def training(self, number_of_iterations):
        errors_series = []
        for i in range(number_of_iterations):
            errors = 0
            for j in range(self.data_nrows):
                row = np.array(self.data.loc[[j]].values[0])
                predicted_value = self.predict( row[:-1], row[-1] )
                #editado
                error = row[-1] - predicted_value[-1]
                errors += int(error != 0.0)
            errors_series.append(errors)
        return np.array(errors_series)
    
    def predict(self, input_vector, target_vector):
        output_vector = None
        for layer in self.layers:
            output_vector = layer.calculate_output(input_vector)
            input_vector = output_vector
        self.adjust_weights(target_vector)
        return output_vector
    
    def predict_value(self, input_vector):
        output_vector = None
        for layer in self.layers:
            output_vector = layer.calculate_output(input_vector)
            input_vector = output_vector
        return output_vector
    
    
    def adjust_weights(self, target_vector):
        flag = True
        errors = []
        deltas_values = []
        deltas_bias_values = []
        
        # Calculo del error para la capa de salida
        error = np.subtract(target_vector, self.layers[-1].output)
        
        delta_out = error * sigmoid_derivative(self.layers[-1].output)
        
        gradiente_out = delta_out*self.layers[-1].input
    
        weights_before = self.layers[-1].weights
    
        
        for index in range(len(self.layers[-1].weights)):
            self.layers[-1].weights[index] -= 0.5*gradiente_out[index]
            
        
        for i in list(reversed(range(len(self.layers))))[1:]:
            
            error_hidden = delta_out*weights_before[:-1]
            
            #bien
            
            v_derivate = sigmoid_derivative(self.layers[i].output)
            
            delta_hidden = error_hidden
        
            
            for n in range(len(error_hidden)):
                delta_hidden[n][0] *= v_derivate[n]
                
            #bien
            
            delta_out = delta_hidden
            ###################################
            
            
            gradiente_hidden = delta_hidden * self.layers[i].input
        
                 
            
            
            weights_before = self.layers[i].weights
            #columna de weights resta con fila de gradientes
            
            for j in range(len(self.layers[i].weights)):
                for k in range(len(gradiente_hidden)):
                    self.layers[i].weights[j][k] -= 0.5*gradiente_hidden[k][j]
            #bien

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 -  sigmoid(x))

def relu(x):
    return np.maximum(0, x)
                
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

def preprocesamiento(dataset):
    data_col_names = dataset.columns.values
    for name in data_col_names[:-1]:
        a[name] = preprocessing.scale(a[name])
        a[name] = ((a[name] - a[name].min())/(a[name].max()-a[name].min()))
        
    return dataset

a = sklearn_to_df(load_wine())

b = TruthTableGenerator.generate_table(n_inputs=4, logic_gate='XOR')
perceptron = MLP(data = b,
                 number_of_inputs=4,
                 number_of_hidden_layers=1,
                 number_of_neurons_for_each_layer=[2,1],
                 activation_functions_for_each_layer=['sigmoid', 'sigmoid'])

nrows, ncols = a.shape
print(a)
print("##############333")
print(preprocesamiento(a))
#predicted_value = self.predict( row[:-1], row[-1] )

     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0      14.23        1.71  2.43               15.6      127.0           2.80   
1      13.20        1.78  2.14               11.2      100.0           2.65   
2      13.16        2.36  2.67               18.6      101.0           2.80   
3      14.37        1.95  2.50               16.8      113.0           3.85   
4      13.24        2.59  2.87               21.0      118.0           2.80   
5      14.20        1.76  2.45               15.2      112.0           3.27   
6      14.39        1.87  2.45               14.6       96.0           2.50   
7      14.06        2.15  2.61               17.6      121.0           2.60   
8      14.83        1.64  2.17               14.0       97.0           2.80   
9      13.86        1.35  2.27               16.0       98.0           2.98   
10     14.10        2.16  2.30               18.0      105.0           2.95   
11     14.12        1.48  2.32               16.8   